In [1]:
# Imports
import os
import sys
import cmath
import math
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import ltspice
import sympy as sp
from sympy.utilities.lambdify import lambdify
from scipy import signal
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from matplotlib.ticker import LogLocator

 - For completely independent noise sources, $C_c = 0$ and we combine the mean-square quantities directly:
 
\begin{equation}
e_n^2 = e_{n1}^2 + e_{n2}^2
\end{equation}

<hr>

$\begin{eqnarray}
1.&& \\[0.25em]
&& Z_f &=& R_f \parallel C_f \\[0.5em]
&& &=& \frac{1}{R_f}+  sC_f \\[0.5em]
&& &=& \frac{1+sR_f C_f}{R_f} \\[0.5em]
&& Z_c &=& \frac{1}{sC_{in}}
\end{eqnarray}$


$\begin{eqnarray}
2.&& \\[0.25em]
&& V_{out} &=& (V^+-V^-) \cdot A_{OL}(f)\Bigg|_{V^+=0} = -V^- \cdot A_{OL}(f)  \\[0em]
& &&  \text{and} \\[0.5em]
&& V_{out} &=& V^- - i_{Z_F} Z_f \\[0.5em]
& &&  \text{then} \\[0.5em]
&& V_{out} &=& -\frac{V_{out}}{A_{OL}(f)}-i_{Z_F} Z_f \\[0.5em]
\end{eqnarray}$

$\begin{eqnarray}
3.&& \\[0.25em]
&& i_{Z_F} &=& i_{in} - \frac{V^-}{Z_{C_{in}}}  \\[0.5em]
&& &=& i_{in} + \frac{V_{out}}{A_{OL}(f)\cdot Z_{C_{in}}}
\end{eqnarray}$


$\begin{eqnarray}
4.&& \\[0.25em]
&& -V_{out} &=& \frac{V_{out}}{A_{OL}(f)} + Z_f(i_{in} + \frac{V_{out}}{A_{OL}(f)\cdot Z_{C_{in}}}) \\[0.5em]
&& Z_f i_{in} &=& \frac{V_{out}}{A_{OL}(f)} + Z_f(\frac{V_{out}}{A_{OL}(f)\cdot Z_{C_{in}}}) + V_{out}\\[0.5em]
&& Z_f i_{in} &=& V_{out}(\frac{1}{A_{OL}(f)} + \frac{Z_f}{A_{OL}(f)\cdot Z_{C_{in}}} + 1)\\[0.5em]
&& \frac{V_{out}}{i_{in}} &=& \frac{Z_f}{\frac{1}{A_{OL}(f)} + \frac{Z_f}{A_{OL}(f)\cdot Z_{C_{in}}} + 1} \\[0.5em]
&&  &=& \frac{Z_f A_{OL}(f)}{1 + \frac{Z_f}{Z_{C_{in}}} + A_{OL}(f)} \\[0.5em]
&&  &=& Z_f \frac{A_{OL}(f)}{1 + A_{OL}(f) + \frac{Z_f}{Z_{C_{in}}}} \\[0.5em]
&&  &=& \frac{1+sR_f C_f}{R_f} \frac{A_{OL}(f)}{1 + A_{OL}(f) + \frac{\frac{1+sR_f C_f}{R_f}}{\frac{1}{sC_{in}}}} \\[0.5em]
&&  &=& \frac{1+sR_f C_f}{R_f} \frac{A_{OL}(f)}{1 + A_{OL}(f) +\frac{(1+sR_f C_f)sC_{in}}{R_f}} \\[0.5em]
\end{eqnarray}$

$\begin{eqnarray}
\text{Check}.&& \\[0.25em]
&& \frac{V_{out}}{i_{in}} &=& Z_f \frac{A_{OL}(f)}{1 + A_{OL}(f) + \frac{Z_f}{Z_{C_{in}}}} \Bigg|_{C_f=0} \\[0.5em]
&& &=& R_f\cdot \frac{A_{OL}(f)}{1+A_{OL}(f)+\frac{R_f}{Z_{C_{in}}}} \\[0.5em]
&& &=& R_f\cdot \frac{A_{OL}(f)}{1+A_{OL}(f)+sC_{in}R_f} \\[0.5em]
\end{eqnarray}$

<hr>

$\begin{eqnarray}
1.&& \\[0.25em]
&& Z_f &=& R_f \parallel C_f \\[0.5em]
&& &=& (\frac{1}{R_f}+  sC_f)^{-1} \\[0.5em]
&& &=& \frac{R_f}{1+sR_f C_f} \\[0.5em]
&& Z_c &=& \frac{1}{sC_{in}}
\end{eqnarray}$


$\begin{eqnarray}
2.&& \\[0.25em]
&& V_{out} &=& (V^+-V^-) \cdot A_{OL}(f)\Bigg|_{V^+=0} = -V^- \cdot A_{OL}(f)  \\[0em]
& &&  \text{and} \\[0.5em]
&& V_{out} &=& V^- - i_{Z_F} Z_f \\[0.5em]
& &&  \text{then} \\[0.5em]
&& V_{out} &=& -\frac{V_{out}}{A_{OL}(f)}-i_{Z_F} Z_f \\[0.5em]
\end{eqnarray}$

$\begin{eqnarray}
3.&& \\[0.25em]
&& i_{Z_F} &=& i_{in} - \frac{V^-}{Z_{C_{in}}}  \\[0.5em]
&& &=& i_{in} + \frac{V_{out}}{A_{OL}(f)\cdot Z_{C_{in}}}
\end{eqnarray}$


$\begin{eqnarray}
4.&& \\[0.25em]
&& -V_{out} &=& \frac{V_{out}}{A_{OL}(f)} + Z_f(i_{in} + \frac{V_{out}}{A_{OL}(f)\cdot Z_{C_{in}}}) \\[0.5em]
&& Z_f i_{in} &=& \frac{V_{out}}{A_{OL}(f)} + Z_f(\frac{V_{out}}{A_{OL}(f)\cdot Z_{C_{in}}}) + V_{out}\\[0.5em]
&& Z_f i_{in} &=& V_{out}(\frac{1}{A_{OL}(f)} + \frac{Z_f}{A_{OL}(f)\cdot Z_{C_{in}}} + 1)\\[0.5em]
&& \frac{V_{out}}{i_{in}} &=& \frac{Z_f}{\frac{1}{A_{OL}(f)} + \frac{Z_f}{A_{OL}(f)\cdot Z_{C_{in}}} + 1} \\[0.5em]
&&  &=& \frac{Z_f A_{OL}(f)}{1 + \frac{Z_f}{Z_{C_{in}}} + A_{OL}(f)} \\[0.5em]
&&  &=& Z_f \frac{A_{OL}(f)}{1 + A_{OL}(f) + \frac{Z_f}{Z_{C_{in}}}} \\[0.5em]
&&  &=& Z_f \frac{1}{1 + \frac{1 + \frac{Z_f}{Z_{C_{in}}}}{A_{OL}(f)}} \\[0.5em]
&&  &=& Z_f \frac{1}{1 + \frac{1}{\beta A_{OL}(f)}} &
\text{where } \frac{1}{\beta} =  1 + \frac{Z_f}{Z_{C_{in}}} =1 + \frac{(1+sR_f C_f)sC_{in}}{R_f} = \frac{C_f C_{in} s^2}{R_f} + \frac{C_{in} s}{R_f} + 1\\[0.5em]
\end{eqnarray}$

$\begin{eqnarray}
\text{Check}.&& \\[0.25em]
&& \frac{V_{out}}{i_{in}} &=& Z_f \frac{A_{OL}(f)}{1 + A_{OL}(f) + \frac{Z_f}{Z_{C_{in}}}} \Bigg|_{C_f=0} \\[0.5em]
&& &=& R_f\cdot \frac{A_{OL}(f)}{1+A_{OL}(f)+\frac{R_f}{Z_{C_{in}}}} \\[0.5em]
&& &=& R_f\cdot \frac{A_{OL}(f)}{1+A_{OL}(f)+sC_{in}R_f} \\[0.5em]
\end{eqnarray}$

<hr>

$\begin{eqnarray}
&& A_{OL}(f) &=& \frac{A_0}{1+\frac{s}{s_p}} & \text{where } A_0 = R_f \\[0.5em]
&& |s_p| &=& \omega_p = w_0 \\[0.5em] 
&& \omega_u &=& A_0 \omega_p \\[0.5em]
&& A_{CL} &=& Z_f \frac{A_{OL}(f)}{1 + A_{OL}(f) + \frac{Z_f}{Z_{C_{in}}}} \\[0.5em]
&& &=& Z_f \frac{\frac{A_0}{1+\frac{s}{s_p}}}{1 +\frac{A_0}{1+\frac{s}{s_p}} + \frac{Z_f}{Z_{C_{in}}}} \\[0.5em]
&& &=& Z_f \cdot \frac{A_0}{1+\frac{s}{s_p}} \cdot \frac{1}{1 +\frac{A_0}{1+\frac{s}{s_p}} + \frac{Z_f}{Z_{C_{in}}}} \\[0.5em]
&& &=& Z_f \cdot \frac{A_0}{1+\frac{s}{s_p}} \cdot \frac{1+\frac{s}{s_p}}{1+\frac{s}{s_p} + A_0 + \frac{Z_f}{Z_{C_{in}}}(1+\frac{s}{s_p})} \\[0.5em]
&& &=& Z_f \cdot A_0 \cdot \frac{1}{1+\frac{s}{s_p} + A_0 + \frac{Z_f}{Z_{C_{in}}}(1+\frac{s}{s_p})} \\[0.5em]
&& &=& Z_f \cdot A_0 \cdot \frac{1}{1+ (A_0 + \frac{Z_f}{Z_{C_{in}}})+ \frac{s}{s_p} (1+\frac{Z_f}{Z_{C_{in}}})} \\[0.5em]
&& &=& Z_f \cdot A_0 \cdot \frac{Z_{C_{in}}}{Z_{C_{in}}+ A_0 Z_{C_{in}} + Z_f + \frac{s}{s_p} (Z_{C_{in}}+Z_f)} \\[0.5em]
&& &=& Z_f \cdot A_0 \cdot \frac{Z_{C_{in}}}{Z_{C_{in}}+ A_0 Z_{C_{in}} + Z_f + \frac{s}{s_p} (Z_{C_{in}}+Z_f)} \\[0.5em]
\end{eqnarray}$

<hr>

<center><img src="img/TIA_Cin.png" width=600 /></center>

 - With the opamp's noise voltage modeled as $e_n$ in series with the non-inverting terminal, feedback forces $e_n$ to appear at the inverting terminal
 
 - The resulting noise current density through $C_{in}$ is $i_{nC} = e_n\cdot 2\pi f C_{in}$
 
 - That is, the amplifier's voltage noise creates a noise current proportional to $C_{in}$ that increases with frequency
 
 - For example, at $1\text{ MHz}$, a noise voltage of $10\text{nV}/\sqrt{\text{Hz}}$ and an input capacitance of $5\text{pF}$ amount to a noise current density of $0.4\text{pA}/\sqrt{\text{Hz}}$
 
 <hr>

The transfer function of a second-order low-pass filter can be written as

\begin{equation}
H(s)={\frac {\omega _{0}^{2}}{s^{2}+2\zeta \omega _{0}\cdot s+\omega _{0}^{2}}}
\end{equation}

where $\omega_0$ is the resonant frequency and $\zeta$ is the damping factor.

In [2]:
s,s_p,A_0,A_f,Z_f,beta,R_f,C_f,Z_Cf,C_in,Z_Cin = sp.symbols('s,s_p,A_0,A_f,Z_f,beta,R_f,C_f,Z_Cf,C_in,Z_Cin')

In [3]:
Z_Cin = 1/(s*C_in)
Z_Cf = 1/(s*C_f)
Z_f = parallel(R_f,Z_Cin)
beta = 1/(1+ Z_f/(Z_Cin))
A_f = A_0/(1+s/s_p)
#sp.simplify(Z_f * A_0 * Z_Cin/(Z_Cin + A_0*Z_Cin + Z_f + (s/s_p)*(Z_Cin+Z_f)))
#sp.simplify(Z_f/(1 + 1/(beta*A_f)))
#sp.simplify(A_f*R_f/(1+s*R_f*(C_in+C_f)+A_f*(1+s*R_f*C_f)))

NameError: name 'parallel' is not defined